# DDPLKO Moduł 7 - praca domowa - HPO

To siódma praca domowa w Programie szkoleniowym Klasyfikacja obrazu od Deep Drive PL

Twoim zadaniem w tym module będzie jendo z 2:

Opcja 1:
- [ ] Na bazie pracy domowej z Transfer learningiem, data augmentation
- [ ] Wykorzystaj HPO (np. Optunę, Hyperopta bądź KerasTuner) by dobrać parametry treningu (optymalizator, LR), data augmentation (zakresy wartości)
- [ ] Udostępnij wykres na Discordzie `#klasyfikacja-wyniki` (val acc - porównanie model przed i po HPO)

Opcja 2 (mniejsza moc obliczeniowa):
- [ ] Na bazie pracy domowej ze zbiorem **QuickDraw**
- [ ] Wykorzystaj HPO (np. Optunę, Hyperopta bądź KerasTuner) by dobrać parametry treningu (optymalizator, LR), i architektury sieci (liczba warstw, dropout, pooling)
- [ ] Pracuj na podzbiorze max 100k przykładów
- [ ] Udostępnij wykres na Discordzie `#klasyfikacja-wyniki` (val acc - porównanie model przed i po HPO)

Możesz extra (czyli opcjonalne rzeczy):
- Spróbować użyć AutoKeras by znaleźć architekturę dla problemu

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set(style='whitegrid',font_scale=1.5)

from keras_tuner import RandomSearch

for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
import tensorflow_datasets as tfds

In [5]:
(ds_train, ds_val),info = tfds.load(
    "resisc45",
    split=["train[:70%]", "train[70%:100%]"],
    as_supervised=True,  # Include labels
    with_info=True
)

print('Train:',len(ds_train))
print('Val:',len(ds_val))

Train: 22050
Val: 9450


In [6]:
img_size = 256
batch_size=32

size = (img_size, img_size)

ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_val = ds_val.map(lambda image, label: (tf.image.resize(image, size), label))
ds_val = ds_val.batch(batch_size=batch_size, drop_remainder=True)

In [7]:
class_names = info.features['label'].names

num_classes = len(class_names)
num_classes

45

# HPO

In [7]:
#from keras_tuner import RandomSearch

def getModel(hp):
          
    l2=tf.keras.regularizers.l2(hp.Float('L2', 1e-8, 0.01, sampling='log'))
    l =[tf.keras.layers.Conv2D(8,(3,3), padding='same', activation='relu',
                               kernel_regularizer=l2, input_shape=(img_size, img_size, 3)),]
    pooling = hp.Int('avg_pooling',0,2)
    bn=hp.Int('BN',0,2)
    
    if bn>0:
        l.append(tf.keras.layers.BatchNormalization())
    for i in range(hp.Int('convs',0,3)):
        if pooling==1:
            l.append(tf.keras.layers.AvgPool2D())
        if pooling==2:
            l.append(tf.keras.layers.MaxPool2D())
        l.append(tf.keras.layers.Conv2D(2**(5+i),(3,3), padding='same', activation='relu',
                        kernel_regularizer=l2))
    if bn==2:
        l.append(tf.keras.layers.BatchNormalization())
    l.append(tf.keras.layers.GlobalAveragePooling2D())
    l.append(tf.keras.layers.Dropout(hp.Float('dropout',0,0.9)))
    l.append(tf.keras.layers.Dense(num_classes, activation='softmax'))
    
    model = tf.keras.models.Sequential(l)
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('lr',0.00001, 0.1, sampling='log')),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [8]:
tuner =RandomSearch(getModel,
                    objective="val_accuracy",
                    max_trials=50,
                    executions_per_trial=1,  # Uśrednianie wyników
                    overwrite=True,
                    directory="keras-tuner-logs",
                    project_name="resisc45")

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
L2 (Float)
{'default': 1e-08, 'conditions': [], 'min_value': 1e-08, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
avg_pooling (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 2, 'step': 1, 'sampling': None}
BN (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 2, 'step': 1, 'sampling': None}
convs (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 3, 'step': 1, 'sampling': None}
dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.9, 'step': None, 'sampling': None}
lr (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.1, 'step': None, 'sampling': 'log'}


In [1]:
from torch.utils import tensorboard

In [2]:
%reload_ext tensorboard
%tensorboard --logdir keras-tuner-logs/tb-logs --port 6001

In [11]:
tuner.search(ds_train, epochs=5, validation_data=(ds_val),
            callbacks=[tf.keras.callbacks.TensorBoard(log_dir='keras-tuner-logs/tb-logs', write_graph=True)])

Trial 50 Complete [00h 15m 46s]
val_accuracy: 0.19735169410705566

Best val_accuracy So Far: 0.5086864233016968
Total elapsed time: 15h 13m 45s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [12]:
models = tuner.get_best_models(num_models=2)

In [18]:
models[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      2336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       7

In [20]:
models[1].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 8)       32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      2336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0

In [13]:
tuner.results_summary()

Results summary
Results in keras-tuner-logs\resisc45
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
L2: 0.00017918884974343376
avg_pooling: 2
BN: 0
convs: 3
dropout: 0.2613001159883856
lr: 0.0009565299993408537
Score: 0.5086864233016968
Trial summary
Hyperparameters:
L2: 3.527369611022989e-07
avg_pooling: 2
BN: 2
convs: 3
dropout: 0.23371613898713336
lr: 0.019694598392422654
Score: 0.4729872941970825
Trial summary
Hyperparameters:
L2: 2.948952727746436e-08
avg_pooling: 0
BN: 0
convs: 3
dropout: 0.19902237187469532
lr: 0.0007624095862106161
Score: 0.4587923586368561
Trial summary
Hyperparameters:
L2: 0.0018141319764431642
avg_pooling: 2
BN: 2
convs: 2
dropout: 0.09140271227428064
lr: 0.001276169814305922
Score: 0.45391950011253357
Trial summary
Hyperparameters:
L2: 0.0008648864369572101
avg_pooling: 2
BN: 2
convs: 3
dropout: 0.3096988778747738
lr: 0.00010862164139984054
Score: 0.4486228823661804
Trial summary
Hyperparameters:
L2: 0.